In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

In [4]:
load_dotenv()

True

In [5]:
llm = ChatOpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    model='gpt-4o-mini',
)

In [6]:
biology_template = '''You are a very smart biology professor. 
You are great at answering questions about biology in a concise and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}
'''

math_template = '''You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}
'''

astronomy_template = '''You are a very good astronomer. You are great at answering astronomy questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}
'''

travel_agent_template = '''You are a very good travel agent with a large amount
of knowledge when it comes to getting people the best deals and recommendations
for travel, vacations, flights and world's best destinations for vacation. 
You are great at answering travel, vacation, flights, transportation, tourist guides questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}
'''

In [7]:
prompt_infos = [
    {
        'name': 'biology',
        'description': 'Good for answering Biology related questions',
        'prompt_template': biology_template
    },
    {
        'name': 'math',
        'description': 'Good for answering math questions',
        'prompt_template': math_template,
    },
    {
        'name': 'astronomy',
        'description': 'Good for answering astronomy questions',
        'prompt_template': astronomy_template,
    },
    {
        'name': 'travel_agent',
        'description': 'Good for answering travel, tourism and vacation questions',
        'prompt_template': travel_agent_template,
    },
]

In [13]:
destination_chains = {}

for info in prompt_infos:
    name = info['name']
    prompt_template = info['prompt_template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    # chain = prompt | llm
    destination_chains[name] = chain
    
destination_chains

{'biology': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="You are a very smart biology professor. \nYou are great at answering questions about biology in a concise and easy to understand manner. \nWhen you don't know the answer to a question you admit that you don't know.\n\nHere is a question:\n{input}\n"), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x107441f90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x107442990>, root_client=<openai.OpenAI object at 0x1074411d0>, root_async_client=<openai.AsyncOpenAI object at 0x1074420d0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'math': L

In [ ]:
# Default chain
default_prompt = ChatPromptTemplate.from_template('{input}')

default_chain = LLMChain(llm=llm, prompt=default_prompt)

LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x107441f90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x107442990>, root_client=<openai.OpenAI object at 0x1074411d0>, root_async_client=<openai.AsyncOpenAI object at 0x1074420d0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={})

In [ ]:
# Setup router
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

destinations = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]
destinations_str = '\n'.join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

LLMRouterChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, output_parser=RouterOutputParser(), partial_variables={}, template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inp

In [22]:
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

/var/folders/fl/gms0qrfn1vb9675l1rjr6x2r0000gn/T/ipykernel_54867/1421187625.py:3: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [26]:
# Test
# question = 'I need to go to Kenya for vacation. Can you help me plan a trip for a family of 4?'
# question = 'How to calculate square root of 2?'
question = 'Tell me some big secret about the universe and galaxies'

response = chain.run(question)
print(response)



> Entering new MultiPromptChain chain...
astronomy: {'input': 'Tell me some big secrets about the universe and galaxies.'}
> Finished chain.
Exploring the universe and galaxies is filled with fascinating discoveries and mysteries. Let’s break down some "big secrets" into component parts to better understand them. 

### 1. **The Expansion of the Universe**
   - **Component Part**: Redshift
     - **Explanation**: The light from distant galaxies is redshifted, meaning it shifts to longer wavelengths as the universe expands. This was first observed by Edwin Hubble.
   - **Connection**: The redshift not only indicates that galaxies are moving away from us, but also helps astronomers calculate the rate of expansion, leading to the understanding of the "Hubble Constant."

### 2. **Dark Matter**
   - **Component Part**: Gravitational effects
     - **Explanation**: Unable to be seen or detected directly, dark matter influences the motion of galaxies and clusters through its gravitational pu